In [1]:
import anndata
import numpy as np
import scvelo as scv
import scanpy as sc
import sys
import pandas as pd
import torch
import os.path
sys.path.append('../../')
import velovae as vv
import matplotlib.pyplot as plt
import umap
from scipy.io import mmwrite
from numpy import savetxt
# import cProfile

In [2]:
path = "/gpfs/accounts/welchjd_root/welchjd0/shared_data/stereo_seq_newCopy/E12.5/E1S1_binned_velocity/"
save_path = "/nfs/turbo/umms-welchjd/yichen/data/scRNA/CytoSignal_Jialin/output"
figure_path_base = f"/scratch/welchjd_root/welchjd/shared_data"

In [ ]:
adata = anndata.read_h5ad(f"{save_path}/stereo_seq_embryo_d12/embryo_d12.h5ad")

In [ ]:
genes_df = pd.read_csv("/nfs/turbo/umms-welchjd/yichen/data/scRNA/CytoSignal_Jialin/intr.genes.list.csv", header=None)
genes_retain = genes_df[0].to_numpy()

# Brain

In [3]:
data_name = "stereo_seq_embryo_d12_brain"

In [ ]:
adata_brain = adata[adata.obs['clusters']=='Brain']

## Preprocessing

In [ ]:
vv.preprocess(adata_brain, 
              n_gene=1000, 
              min_count_per_cell=0,
              compute_umap=True, 
              resolution=0.5,
              genes_retain=genes_retain)

In [ ]:
print(len(set(genes_retain).intersection(set(adata_brain.var_names))))

In [ ]:
adata_brain.write_h5ad(f"{save_path}/{data_name}/embryo_d12_brain_pp.h5ad")

## Train Model

In [4]:
adata_brain = anndata.read_h5ad(f"{save_path}/{data_name}/embryo_d12_brain_pp.h5ad")

In [6]:
figure_path = f"{figure_path_base}/{data_name}/VAE"
model_path = f"{save_path}/{data_name}/VAE"
data_path = f"{save_path}/{data_name}"
torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata_brain, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0')
std_u = np.std(adata_brain.layers['Mu'],0)
std_s = np.std(adata_brain.layers['Ms'],0)
mask = (std_u > 0.1) & (std_s > 0.1)
gene_plot = np.random.choice(adata_brain.var_names[mask], 4)
config = {
}
vae.train(adata_brain, config=config, gene_plot=gene_plot, plot=False, figure_path=figure_path, embed="pos")
vae.save_model(model_path, 'velovae', 'velovae')
vae.save_anndata(adata_brain, 'velovae', data_path, file_name="adata_brain_out.h5ad")

Estimating ODE parameters...


  0%|          | 0/1469 [00:00<?, ?it/s]

Detected 279 velocity genes.
Estimating the variance...


  0%|          | 0/1469 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.64, 0.7522774429271357), (0.36, 0.12022164866515708)
(0.52, 0.7486019979175588), (0.48, 0.13050691493843053)
(0.67, 0.39816690366436347), (0.33, 0.7525797476391921)
KS-test result: [0. 0. 0.]
Initial induction: 761, repression: 708/1469
Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


...

Percentage of Invalid Sets: 0.043
Average Set Size: 231
Finished. Actual Time:   0 h :  0 m : 11 s
*********       Round 5: Early Stop Triggered at epoch 1566.       *********
Change in noise variance: 0.015306109562516212
Change in x0: 0.13562320030887334
*********             Velocity Refinement Round 6              *********
Cell-wise KNN Estimation.


  0%|          | 0/11525 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.043
Average Set Size: 231
Finished. Actual Time:   0 h :  0 m : 11 s
*********       Round 6: Early Stop Triggered at epoch 1600.       *********
Change in noise variance: 0.005762095563113689
Change in x0: 0.12819582387660072
*********             Velocity Refinement Round 7              *********
Cell-wise KNN Estimation.


  0%|          | 0/11525 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.043
Average Set Size: 231
Finished. Actual Time:   0 h :  0 m : 11 s
*********       Round 7: Early Stop Triggered at epoch 1632.       *********
Change in noise variance: 0.0090827327221632
Change in x0: 0.1463530272521835
*********             Velocity Refinement Round 8              *********
Stage 2: Early Stop Triggered at round 7.
*********              Finished. Total Time =   0 h : 24 m : 56 s             *********
Final: Train ELBO = 12232.753,	Test ELBO = 12235.343


In [7]:
figure_path = f"{figure_path_base}/{data_name}/FullVB"
model_path = f"{save_path}/{data_name}/FullVB"
data_path = f"{save_path}/{data_name}"

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata_brain, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             full_vb=True)
gene_plot = np.random.choice(adata_brain.var_names, 4)

vae.train(adata_brain, gene_plot=gene_plot, plot=False, figure_path=figure_path, embed="pos")
vae.save_model(model_path, 'fullvb', 'fullvb')
vae.save_anndata(adata_brain, 'fullvb', data_path, file_name="adata_brain_out.h5ad")

Estimating ODE parameters...


  0%|          | 0/1469 [00:00<?, ?it/s]

Detected 279 velocity genes.
Estimating the variance...


  0%|          | 0/1469 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.64, 0.7522774429271357), (0.36, 0.12022164866515708)
(0.52, 0.7486019979175588), (0.48, 0.13050691493843053)
(0.67, 0.39816690366436347), (0.33, 0.7525797476391921)
KS-test result: [0. 0. 0.]
Initial induction: 761, repression: 708/1469
Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


...

Percentage of Invalid Sets: 0.040
Average Set Size: 231
Finished. Actual Time:   0 h :  0 m : 11 s
*********       Round 18: Early Stop Triggered at epoch 1729.       *********
Change in noise variance: 0.0
Change in x0: 0.13055984911466537
*********             Velocity Refinement Round 19              *********
Cell-wise KNN Estimation.


  0%|          | 0/11525 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.040
Average Set Size: 231
Finished. Actual Time:   0 h :  0 m : 11 s
*********       Round 19: Early Stop Triggered at epoch 1739.       *********
Change in noise variance: 0.0
Change in x0: 0.12849785878388978
*********             Velocity Refinement Round 20              *********
Cell-wise KNN Estimation.


  0%|          | 0/11525 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.040
Average Set Size: 231
Finished. Actual Time:   0 h :  0 m : 11 s
*********       Round 20: Early Stop Triggered at epoch 1755.       *********
Change in noise variance: 0.0
Change in x0: 0.12734613697807753
*********              Finished. Total Time =   0 h : 33 m : 13 s             *********
Final: Train ELBO = 8797.238,	Test ELBO = 8806.635


In [ ]:
adata_brain = anndata.read_h5ad(f"{save_path}/{data_name}/adata_brain_out.h5ad")

In [8]:
gene_plot = adata_brain.var_names
vv.post_analysis(adata_brain,
                 'eval',
                 ['VeloVAE','FullVB'],
                 ['velovae','fullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(2,5),
                 embed='pos',
                 plot_type=['signal','time','stream'],
                 save_path=f"{figure_path_base}/{data_name}",
                 sparsity_correction=True)

Computing velocity embedding using scVelo
computing velocity graph (using 3/32 cores)


  0%|          | 0/11525 [00:00<?, ?cells/s]

    finished (0:01:43) --> added 
    'velovae_velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:03) --> added
    'velovae_velocity_pos', embedded velocity vectors (adata.obsm)
Computing velocity embedding using scVelo
computing velocity graph (using 3/32 cores)


  0%|          | 0/11525 [00:00<?, ?cells/s]

    finished (0:01:36) --> added 
    'fullvb_velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:03) --> added
    'fullvb_velocity_pos', embedded velocity vectors (adata.obsm)
---     Computing Peformance Metrics     ---
Dataset Size: 11525 cells, 1469 genes
---   Plotting  Results   ---
saving figure to file /scratch/welchjd_root/welchjd/shared_data/stereo_seq_embryo_d12_brain/eval_velovae_stream.png
saving figure to file /scratch/welchjd_root/welchjd/shared_data/stereo_seq_embryo_d12_brain/eval_fullvb_stream.png


,VeloVAE,FullVB
MSE Train,0.869,1.609
MSE Test,0.830,1.580
MAE Train,0.105,0.154
MAE Test,0.105,0.152
LL Train,12256.787,8821.289
LL Test,12258.849,8838.817
Training Time,1496.458,1993.851
In-Cluster Coherence,0.731,0.966


In [ ]:
genes = adata_brain.var.index.to_numpy()
barcodes = adata_brain.obs.index.to_numpy()
pd.DataFrame(genes).to_csv(f"{data_path}/genes_{data_name}.csv")
pd.DataFrame(barcodes).to_csv(f"{data_path}/barcodes_{data_name}.csv")
print(data_path)
for key in ['velovae', 'fullvb']:
    time = adata_brain.obs[f"{key}_time"].to_numpy()
    pd.DataFrame(time).to_csv(f"{data_path}/time_{data_name}_{key}.csv")
    mmwrite(f"{data_path}/vu_{data_name}_{key}.mtx", adata_brain.layers[f"{key}_velocity_u"])
    mmwrite(f"{data_path}/vs_{data_name}_{key}.mtx", adata_brain.layers[f"{key}_velocity"])

# Jaw and Tooth

In [ ]:
data_name = "stereo_seq_embryo_d12_jaw_tooth"

In [ ]:
adata_jt = adata[adata.obs['clusters']=='Jaw and tooth']

In [ ]:
vv.preprocess(adata_jt, 
              n_gene=1000, 
              min_count_per_cell=0,
              compute_umap=True, 
              resolution=0.5,
              genes_retain=genes_retain)

In [ ]:
print(len(set(genes_retain).intersection(set(adata_jt.var_names))))

In [ ]:
adata_jt.write_h5ad(f"{save_path}/{data_name}/embryo_d12_jaw_tooth_pp.h5ad")

In [ ]:
adata_jt = anndata.read_h5ad(f"{save_path}/{data_name}/embryo_d12_jaw_tooth_pp.h5ad")

In [ ]:
figure_path = f"{figure_path_base}/{data_name}/VAE"
model_path = f"{save_path}/{data_name}/VAE"
data_path = f"{save_path}/{data_name}"
torch.manual_seed(2023)
np.random.seed(2023)
vae = vv.VAE(adata_jt, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             reverse_gene_mode=False)
std_u = np.std(adata_jt.layers['Mu'],0)
std_s = np.std(adata_jt.layers['Ms'],0)
mask = (std_u > 0.1) & (std_s > 0.1)
gene_plot = np.random.choice(adata_jt.var_names[mask], 4)
config = {
    'batch_size':64
}
vae.train(adata_jt, config=config, gene_plot=gene_plot, plot=False, figure_path=figure_path, embed="pos")
vae.save_model(model_path, 'velovae', 'velovae')
vae.save_anndata(adata_jt, 'velovae', data_path, file_name="adata_jt_out.h5ad")

In [ ]:
figure_path = f"{figure_path_base}/{data_name}/FullVB"
model_path = f"{save_path}/{data_name}/FullVB"
data_path = f"{save_path}/{data_name}"

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata_jt, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             full_vb=True,
             reverse_gene_mode=False)
gene_plot = np.random.choice(adata_jt.var_names, 4)
config = {
    'batch_size':64
}
vae.train(adata_jt, config=config, gene_plot=gene_plot, plot=False, figure_path=figure_path, embed="pos")
vae.save_model(model_path, 'fullvb', 'fullvb')
vae.save_anndata(adata_jt, 'fullvb', data_path, file_name="adata_jt_out.h5ad")

In [ ]:
gene_plot = adata_jt.var_names
vv.post_analysis(adata_jt,
                 'eval',
                 ['VeloVAE', 'FullVB'],
                 ['velovae', 'fullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(2,5),
                 embed='umap',
                 plot_type=['signal', 'time', 'stream'],
                 save_path=f"{figure_path_base}/{data_name}",
                 sparsity_correction=True)

In [ ]:
genes = adata_jt.var.index.to_numpy()
barcodes = adata_jt.obs.index.to_numpy()
pd.DataFrame(genes).to_csv(f"{data_path}/genes_{data_name}.csv")
pd.DataFrame(barcodes).to_csv(f"{data_path}/barcodes_{data_name}.csv")
print(data_path)
for key in ['velovae', 'fullvb']:
    time = adata_jt.obs[f"{key}_time"].to_numpy()
    pd.DataFrame(time).to_csv(f"{data_path}/time_{data_name}_{key}.csv")
    mmwrite(f"{data_path}/vu_{data_name}_{key}.mtx", adata_jt.layers[f"{key}_velocity_u"])
    mmwrite(f"{data_path}/vs_{data_name}_{key}.mtx", adata_jt.layers[f"{key}_velocity"])

# GI tract + Mesentery

In [ ]:
data_name = "stereo_seq_embryo_d12_gi_mensentery"

In [ ]:
np.unique(adata.obs['clusters'])

In [ ]:
adata_gm = adata[(adata.obs['clusters']=='GI tract') | (adata.obs['clusters']=='Mesentery')]

In [ ]:
vv.preprocess(adata_gm, 
              n_gene=1000, 
              min_count_per_cell=0,
              compute_umap=True, 
              resolution=0.5,
              genes_retain=genes_retain,
              perform_clustering=True)

In [ ]:
scv.pl.scatter(adata_gm)

In [ ]:
figure_path = f"{figure_path_base}/{data_name}/VAE"
model_path = f"{save_path}/{data_name}/VAE"
data_path = f"{save_path}/{data_name}"
torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata_gm, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             reverse_gene_mode=False)
std_u = np.std(adata_gm.layers['Mu'],0)
std_s = np.std(adata_gm.layers['Ms'],0)
mask = (std_u > 0.1) & (std_s > 0.1)
gene_plot = np.random.choice(adata_gm.var_names[mask], 4)
config = {
    'batch_size':64
}
vae.train(adata_gm, config=config, gene_plot=gene_plot, plot=False, figure_path=figure_path, embed="pos")
vae.save_model(model_path, 'velovae', 'velovae')
vae.save_anndata(adata_gm, 'velovae', data_path, file_name="adata_gm_out.h5ad")

In [ ]:
figure_path = f"{figure_path_base}/{data_name}/FullVB"
model_path = f"{save_path}/{data_name}/FullVB"
data_path = f"{save_path}/{data_name}"
torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata_gm, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             full_vb=True,
             reverse_gene_mode=False)
std_u = np.std(adata_gm.layers['Mu'],0)
std_s = np.std(adata_gm.layers['Ms'],0)
mask = (std_u > 0.1) & (std_s > 0.1)
gene_plot = np.random.choice(adata_gm.var_names[mask], 4)
config = {
    'batch_size':64
}
vae.train(adata_gm, config=config, gene_plot=gene_plot, plot=False, figure_path=figure_path, embed="pos")
vae.save_model(model_path, 'fullvb', 'fullvb')
vae.save_anndata(adata_gm, 'fullvb', data_path, file_name="adata_gm_out.h5ad")

In [ ]:
gene_plot = adata_gm.var_names
vv.post_analysis(adata_gm,
                 'eval',
                 ['VeloVAE', 'FullVB'],
                 ['velovae', 'fullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(2,5),
                 embed='umap',
                 plot_type=['signal', 'time', 'stream'],
                 save_path=f"{figure_path_base}/{data_name}",
                 sparsity_correction=True)

In [ ]:
genes = adata_gm.var.index.to_numpy()
barcodes = adata_gm.obs.index.to_numpy()
pd.DataFrame(genes).to_csv(f"{data_path}/genes_{data_name}.csv")
pd.DataFrame(barcodes).to_csv(f"{data_path}/barcodes_{data_name}.csv")
print(data_path)
for key in ['velovae', 'fullvb']:
    time = adata_jt.obs[f"{key}_time"].to_numpy()
    pd.DataFrame(time).to_csv(f"{data_path}/time_{data_name}_{key}.csv")
    mmwrite(f"{data_path}/vu_{data_name}_{key}.mtx", adata_jt.layers[f"{key}_velocity_u"])
    mmwrite(f"{data_path}/vs_{data_name}_{key}.mtx", adata_jt.layers[f"{key}_velocity"])